In [351]:
import requests
import urllib
import html2text as ht
import re
import pandas as pd
import numpy as np
import json
import string
from bs4 import BeautifulSoup
from collections import Counter


## Scrape Billboard for song titles

In [260]:
path = "https://www.billboard.com/charts/year-end/2017/hot-100-songs"

In [261]:
def clean_text(text):
    bad = {"\\\\n+": "\n", 
           "\n+": "\n", 
           "\n": " - ",
           "\[.{0,}?\]": "", 
           "\(.{0,}?\)": ""}
    
    for x in bad:
        text = re.sub(x, bad[x], text)
        
    return text

def find_songs(text):
    
    s = "[0-9]{1,3} - !.+? !"
    songs = [x.group() for x in list(re.finditer(s, text))]
    return songs

def get_titles(songs):
    repl = {"- ! -": ":::", "- \*.+": "", "- !": ""}
    for i in range(len(songs)):
        for j in repl:
            songs[i] = re.sub(j, repl[j], songs[i])
        songs[i] = songs[i].strip()
        
    return songs

In [515]:
sub = "SELECT A YEAR"
page = requests.get(path)
filler = [" and ", "featuring", " & ", " x ", " / "]

data = ht.html2text(str(page.content))
data = clean_text(data)
data = data[re.search(sub, data).end():]

songs = find_songs(data)
songs = np.array([[y.strip() for y in re.split("\:\:\:| - ", x)] for x in get_titles(songs)])
# take out filler words, such as and, &, featuring
for i in range(len(songs[:, 2])):
    for j in filler:
        if re.search(j, songs[i, 2], flags = re.I) != None:
            songs[i, 2] = ", ".join(x.strip() for x in re.split(j, songs[i, 2], flags = re.I))
df = pd.DataFrame({"Rank": songs[:, 0], "Name": songs[:, 1], "Artists": songs[:, 2]})

In [516]:
def get_lyric_link(artists, name, both = False):
    

    headers = {"Authorization": "Bearer " + client_token}
    link = "https://api.genius.com/search?q="
    space = "%20"
    name_repl = {"Beyonce": "Beyoncé", 
                 "Amine": "Aminé",
                 "D.R.A.M.": "DRAM",
                 "$ign": "\$ign"}

    artists = re.sub(",", "", artists)
    name = re.sub(",", "", name)
    if both:
        page = requests.get(link + re.sub(" ", space, name) +
                            space + re.sub(" ", space, artists), headers = headers)
    else:
        page = requests.get(link + re.sub(" ", space, name), headers = headers)
    page = json.loads(page.content)["response"]["hits"] #data
    check = [re.sub(",", "", x) for x in artists.split(" ") + name.split(" ") if x not in string.punctuation]
    for i in name_repl:
        if i in check:
            check.remove(i)
            check.append(name_repl[i])
    top = []
    #print(page, check)
    
    #print(link + re.sub(" ", space, name) +
    #                    space + re.sub(" ", space, artists), check, page)
    
    
    if len(page) == 1:
        return page[0]["result"]["path"]
    else:
        for i in range(len(page)):
            c = 0
            for j in check:
                if re.search(j, page[i]["result"]["full_title"], flags = re.I | re.S) != None:
                    c += 1
                    #print(c)
                    #print(page[i]["result"]["full_title"])
            #print(c, len(check))
            if c == len(check):
                return page[i]["result"]["path"]
            #for j in 
            #try:
            #    top.append(page[i]["result"]["stats"]["pageviews"])
            #except KeyError:
            #    top.append(-1)
        
    if both == False:
        return get_lyric_link(artists, name, True)
    
    top = []
    #if here, find most popular song
    for j in range(len(page)):
        try:
            top.append(page[i]["result"]["stats"]["pageviews"])
        except KeyError:
            top.append(-1)
    print(top)
    return page[np.argmax(top)]["result"]["path"]

    #
        
    #return None#page[np.argmax(top)]["result"]["path"]

In [535]:
def scrape_lyrics(row, snip):
    
    base_link = "https://genius.com"
    
    genre_sub = {"\"": " ", "genius": ""}
    lyric_regex = row["Name"] + ".{0,20}?Lyrics.+?More on Genius"
    genre_regex = "genres\":\[.+?\]"
    track_info_regex = "\"{}".format(row["Name"]) + ".{0,20}?\".{0,20}?track info.+?remixed by"
    track_info_regex2 = "\"{}".format(row["Name"]) + ".{0,20}?\".{0,20}?track info.+?cover by"
    date_regex = "release date.{0,15}?20[0-9][0-9]"
    write_regex = "Written By\n+.+?\n"
    data = BeautifulSoup(requests.get(base_link + snip).content, "lxml").get_text()

    #subset data
    data = data[re.search(base_link + snip, data).end():]
    #print(data)
    lyrics = re.search(lyric_regex, data, flags = re.I | re.S).group()[:-15]
    genre = re.search("\[.+?\]", re.search(genre_regex, data, flags = re.I | re.S).group()).group()[1:-1]
    for k in genre_sub:
        genre = re.sub(k, genre_sub[k], genre, flags = re.I)
    #try:
    #    track_info = re.search(track_info_regex, data, flags = re.I | re.S).group()
    #except AttributeError:
    #    track_info = re.search(track_info_regex2, data, flags = re.I | re.S).group()
    date = re.split("\n+", re.search(date_regex, data, flags = re.I | re.S).group())[-1]
    write = [x for x in re.split("\n+", re.search(write_regex, data, flags = re.I | re.S).group()) if x != ""][-1]

    return date, genre, write, lyrics

In [536]:
#i = 84
#temp = get_lyric_link(df.iloc[i]["Artists"],  df.iloc[i]["Name"])
#d, g, w, l = scrape_lyrics(df.iloc[i], temp)

In [537]:
dates = []
genre = []
write = []
lyrics = []

for i in range(len(df)):
    temp = get_lyric_link(df.iloc[i]["Artists"],  df.iloc[i]["Name"])
    d, g, w, l = scrape_lyrics(df.iloc[i], temp)
    if i % 20 == 0:
        print(i)
    dates.append(d)
    genre.append(g)
    write.append(w)
    lyrics.append(l)
    

0
20
40
60
80


In [545]:
df["Date"] = dates
df["Genre"] = genre
df["Writing Credits"] = write
df["Lyrics"] = lyrics

In [546]:
df.to_csv("Billboard_2017.csv", header = False)

## Scrape Genius for lyrics, genre, etc.


Song

Artist/Group

Billboard rank [or peak rank?]

Year

Genre [if available]

Writing Credits

Lyrics

